In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
qdrant_api_key = os.getenv("QDRANT_API")
qdrant_endpoint = os.getenv("QDRANT_ENDPOINT")
if not openai_api_key or not qdrant_api_key or not qdrant_endpoint:
    raise ValueError("key not found in environment variables.")
print("all keys loaded successfully.")


all keys loaded successfully.


Temporarly using default logs, later during actual implementation take the logs from the user

In [8]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("../mysql_crash_log.txt")
documents = loader.load()
documents[0].page_content[:500]  # Display the first 500 characters of the document

"Jan 09 13:51:09 deilsy systemd[1]: Starting mysql.service - MySQL Community Server...\nJan 09 13:51:09 deilsy mysql-systemd-start[149476]: MySQL has been frozen to prevent damage to your system. Please see /etc/mysql/FROZEN for help.\nJan 09 13:51:09 deilsy systemd[1]: mysql.service: Control process exited, code=exited, status=1/FAILURE\nJan 09 13:51:09 deilsy systemd[1]: mysql.service: Failed with result 'exit-code'.\nJan 09 13:51:09 deilsy systemd[1]: Failed to start mysql.service - MySQL Communit"

In [41]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)
split_docs

[Document(metadata={'source': '../mysql_crash_log.txt'}, page_content="Jan 09 13:51:09 deilsy systemd[1]: Starting mysql.service - MySQL Community Server...\nJan 09 13:51:09 deilsy mysql-systemd-start[149476]: MySQL has been frozen to prevent damage to your system. Please see /etc/mysql/FROZEN for help.\nJan 09 13:51:09 deilsy systemd[1]: mysql.service: Control process exited, code=exited, status=1/FAILURE\nJan 09 13:51:09 deilsy systemd[1]: mysql.service: Failed with result 'exit-code'.\nJan 09 13:51:09 deilsy systemd[1]: Failed to start mysql.service - MySQL Community Server.\nJan 09 13:51:10 deilsy systemd[1]: mysql.service: Scheduled restart job, restart counter is at 1.\nJan 09 13:51:10 deilsy systemd[1]: Starting mysql.service - MySQL Community Server...\nJan 09 13:51:10 deilsy mysql-systemd-start[149478]: MySQL has been frozen to prevent damage to your system. Please see /etc/mysql/FROZEN for help.\nJan 09 13:51:10 deilsy systemd[1]: mysql.service: Control process exited, code=e

Embedding documents

In [73]:
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, PointStruct, VectorParams
import random

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=openai_api_key
)

collection_name = "mysql_crash_analysis"
url = "http://localhost:6333"

# Create client and delete collection if exists
client = QdrantClient(url=url)
try:
    client.delete_collection(collection_name)
except:
    pass

# Get vector size from embeddings
vector_size = len(embeddings.embed_query("test"))

# Create collection
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)

# Embed and upsert documents
points = []
for idx, doc in enumerate(split_docs):
    embedding = embeddings.embed_query(doc.page_content)
    point = PointStruct(
        id=idx,  # Use simple integer ID
        vector=embedding,
        payload={
            "content": doc.page_content,
            "metadata": doc.metadata
        }
    )
    points.append(point)

# Upsert points in batches
batch_size = 100
for i in range(0, len(points), batch_size):
    batch = points[i:i+batch_size]
    client.upsert(collection_name=collection_name, points=batch)

print("Vectorstore created successfully!")

Vectorstore created successfully!


In [77]:
# Query using REST API directly
import requests

query_text = "why did server failed"
query_embedding = embeddings.embed_query(query_text)

# Use REST API to search
response = requests.post(
    f"{url}/collections/{collection_name}/points/search",
    json={
        "vector": query_embedding,
        "limit": 6,
        "with_payload": True
    }
)

result_data = response.json()

# Format results as documents
docs = []
for point in result_data.get("result", []):
    docs.append({
        "content": point["payload"]["content"],
        "score": point["score"]
    })

print(f"Found {len(docs)} relevant documents:")
for i, doc in enumerate(docs, 1):
    print(f"\n--- Document {i} (Score: {doc['score']:.4f}) ---")
    print(doc["content"][:500])

Found 6 relevant documents:

--- Document 1 (Score: 0.3842) ---
Jan 09 13:51:10 deilsy systemd[1]: mysql.service: Control process exited, code=exited, status=1/FAILURE
Jan 09 13:51:10 deilsy systemd[1]: mysql.service: Failed with result 'exit-code'.
Jan 09 13:51:10 deilsy systemd[1]: Failed to start mysql.service - MySQL Community Server.
Jan 09 13:51:10 deilsy systemd[1]: mysql.service: Scheduled restart job, restart counter is at 2.
Jan 09 13:51:10 deilsy systemd[1]: Starting mysql.service - MySQL Community Server...
Jan 09 13:51:10 deilsy mysql-systemd-st

--- Document 2 (Score: 0.3810) ---
Jan 09 13:51:10 deilsy systemd[1]: Failed to start mysql.service - MySQL Community Server.
Jan 09 13:51:10 deilsy systemd[1]: mysql.service: Scheduled restart job, restart counter is at 3.
Jan 09 13:51:10 deilsy systemd[1]: Starting mysql.service - MySQL Community Server...
Jan 09 13:51:10 deilsy mysql-systemd-start[149482]: MySQL has been frozen to prevent damage to your system. Please see /etc/

In [79]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain_classic.prompts import ChatPromptTemplate

In [16]:
prompt_template = ChatPromptTemplate.from_template(
    """You are a Site Reliability Engineer.

Answer ONLY using the provided logs.
Do NOT use prior knowledge or assumptions.
If the logs do not contain enough information, say exactly: "Insufficient evidence".

Logs:
{context}

Question: {input}

Answer format:
- Root Cause
- Evidence (exact log lines)
- Fix
- Prevention
"""
)


In [80]:
from langchain_openai import ChatOpenAI
import requests

# Initialize LLM
llm = ChatOpenAI(
    api_key=openai_api_key,
    model="gpt-4o",
    temperature=0.2
)

# Define RAG function
def rag_query(query_text):
    # Step 1: Retrieve relevant documents
    query_embedding = embeddings.embed_query(query_text)
    
    response = requests.post(
        f"{url}/collections/{collection_name}/points/search",
        json={
            "vector": query_embedding,
            "limit": 6,
            "with_payload": True
        }
    )
    
    result_data = response.json()
    
    # Extract context from search results
    context_docs = []
    for point in result_data.get("result", []):
        context_docs.append(point["payload"]["content"])
    
    context = "\n\n---\n\n".join(context_docs)
    
    # Step 2: Generate answer using LLM
    prompt = f"""You are a Site Reliability Engineer.

Answer ONLY using the provided logs.
Do NOT use prior knowledge or assumptions.
If the logs do not contain enough information, say exactly: "Insufficient evidence".

Logs:
{context}

Question: {query_text}

Answer format:
- Root Cause
- Evidence (exact log lines)
- Fix
- Prevention"""
    
    from langchain_core.messages import HumanMessage
    
    message = HumanMessage(content=prompt)
    response = llm.invoke([message])
    
    return response.content

# Test the RAG chain
query = "what made my server fails?"
result = rag_query(query)

print(result)

- Root Cause  
  MySQL service was frozen to prevent damage to the system.

- Evidence  
  - "Jan 09 13:51:10 deilsy mysql-systemd-start[149480]: MySQL has been frozen to prevent damage to your system. Please see /etc/mysql/FROZEN for help."
  - "Jan 09 13:51:28 deilsy mysql-systemd-start[149612]: MySQL has been frozen to prevent damage to your system. Please see /etc/mysql/FROZEN for help."

- Fix  
  Review the contents of the file `/etc/mysql/FROZEN` for guidance on resolving the issue that caused MySQL to freeze.

- Prevention  
  Implement monitoring and alerting to detect and address issues that could lead to MySQL being frozen before they occur. Regularly review and maintain the MySQL configuration and system resources to prevent conditions that could trigger a freeze.


In [82]:
# Check available methods
available_methods = [method for method in dir(client) if "search" in method.lower()]
print("Available search methods:", available_methods)

Available search methods: ['search_matrix_offsets', 'search_matrix_pairs']
